In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
train = pd.read_csv('train.csv')
camp_data = pd.read_csv('campaign_data.csv')
coupon = pd.read_csv('coupon_item_mapping.csv')
demo = pd.read_csv('customer_demographics.csv')
trans = pd.read_csv('customer_transaction_data.csv')
item = pd.read_csv('item_data.csv')

### EDA 


### Train Dataset

In [2]:
print(train.head(5))
print('*'*80)
print(train.info())

   id  campaign_id  coupon_id  customer_id  redemption_status
0   1           13         27         1053                  0
1   2           13        116           48                  0
2   6            9        635          205                  0
3   7           13        644         1050                  0
4   9            8       1017         1489                  0
********************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78369 entries, 0 to 78368
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   id                 78369 non-null  int64
 1   campaign_id        78369 non-null  int64
 2   coupon_id          78369 non-null  int64
 3   customer_id        78369 non-null  int64
 4   redemption_status  78369 non-null  int64
dtypes: int64(5)
memory usage: 3.0 MB
None


In [3]:
print(train.campaign_id.unique())
print(" ")
print(len(train.customer_id.unique()))
print(" ")
print(train.redemption_status.unique())
print(" ")
print(train.coupon_id.unique())
print(" ")
print(train.coupon_id.value_counts())

[13  9  8 11 29 30  2  5 12 26  3  4 10  7 28 27  6  1]
 
1428
 
[0 1]
 
[  27  116  635  644 1017  795  444  538  857  559  575 1028  705  517
  893  796  506  689  268    8  674  320  109  278 1031  326  363  911
  383  482  813  118  663  332  500  757 1067   87  905 1114  152   57
  271  413  582  333  485  256  980  700  628  136  750  854  761  828
  881  129  867  709  169 1083  530  822  126  917   22  960   24  772
  603  982  510  132   60 1063  166  679  839  459  658  106  168  473
  627  145    7  667  924  621  467  985  171 1056  890  920  279  948
  368  107   19  800  420  756  783  687  913  725  483  275  788  170
   14  416   54  113  125  902  526  144  554  936  971  872  873  643
  433  131  591  608  723   76  696  544  148   97  819  673  123  490
   55  989   83  437  836  806  891  870  542   64  646  566  520  910
  581  537  656  996 1012  759 1044  616  883  754  430  786  694  119
  481  940  916  348  855  792  968  625  105  470  283  161    6  466
 103

In [4]:
print(train.campaign_id.value_counts())
print('*'*50)
print(train.redemption_status.value_counts())

13    22606
8     22555
30     6467
26     6056
29     3895
9      3168
11     2782
12     2550
5      1827
10     1723
7      1584
4       972
2       768
28      476
3       408
27      324
1       143
6        65
Name: campaign_id, dtype: int64
**************************************************
0    77640
1      729
Name: redemption_status, dtype: int64


### Campaign data


In [5]:
print(camp_data.head(5))
print('*'*80)
camp_data.info()

   campaign_id campaign_type start_date  end_date
0           24             Y   21/10/13  20/12/13
1           25             Y   21/10/13  22/11/13
2           20             Y   07/09/13  16/11/13
3           23             Y   08/10/13  15/11/13
4           21             Y   16/09/13  18/10/13
********************************************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   campaign_id    28 non-null     int64 
 1   campaign_type  28 non-null     object
 2   start_date     28 non-null     object
 3   end_date       28 non-null     object
dtypes: int64(1), object(3)
memory usage: 1.0+ KB


In [6]:
from datetime import datetime as dtime
def convertdate(date):
    result=[]
    for i,dt in enumerate(date):
        dt = dt[:-2]+'20'+dt[-2:]
        dt = dtime.strptime(dt,"%d/%m/%Y")
        result.append(dt)
    return pd.Series(result)
end_date = convertdate(camp_data.end_date)
start_date = convertdate(camp_data.start_date)
camp_data.end_date = end_date
camp_data.start_date = start_date
camp_data.head(5)

,campaign_id,campaign_type,start_date,end_date
0,24,Y,2013-10-21,2013-12-20
1,25,Y,2013-10-21,2013-11-22
2,20,Y,2013-09-07,2013-11-16
3,23,Y,2013-10-08,2013-11-15
4,21,Y,2013-09-16,2013-10-18


In [7]:
func = lambda x: x.days
camp_data['Days']=camp_data.end_date - camp_data.start_date
camp_data['Days'] = camp_data['Days'].apply(func)
camp_data.head(5)

,campaign_id,campaign_type,start_date,end_date,Days
0,24,Y,2013-10-21,2013-12-20,60
1,25,Y,2013-10-21,2013-11-22,32
2,20,Y,2013-09-07,2013-11-16,70
3,23,Y,2013-10-08,2013-11-15,38
4,21,Y,2013-09-16,2013-10-18,32


In [8]:
print(camp_data.campaign_type.unique())
print(camp_data.campaign_type.value_counts())
print('*'*40)
print(camp_data.campaign_id.unique())

['Y' 'X']
Y    22
X     6
Name: campaign_type, dtype: int64
****************************************
[24 25 20 23 21 22 18 19 17 16 13 11 12 10  9  8  7  6  3  5  4  1  2 30
 29 28 27 26]


### Coupon item mapping



In [9]:
print(coupon.head(5))
print("*"*50)
print(coupon.info())

   coupon_id  item_id
0        105       37
1        107       75
2        494       76
3        522       77
4        518       77
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92663 entries, 0 to 92662
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   coupon_id  92663 non-null  int64
 1   item_id    92663 non-null  int64
dtypes: int64(2)
memory usage: 1.4 MB
None


In [10]:
print(len(coupon.item_id.unique()))
print(len(coupon.coupon_id.unique()))


36289
1116


### Customer Transaction Data 

In [11]:
trans.head(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


In [12]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1324566 entries, 0 to 1324565
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   date             1324566 non-null  object 
 1   customer_id      1324566 non-null  int64  
 2   item_id          1324566 non-null  int64  
 3   quantity         1324566 non-null  int64  
 4   selling_price    1324566 non-null  float64
 5   other_discount   1324566 non-null  float64
 6   coupon_discount  1324566 non-null  float64
dtypes: float64(3), int64(3), object(1)
memory usage: 70.7+ MB


In [13]:
print(item.head(5))
print('*'*50)
print(item.info())

   item_id  brand   brand_type       category
0        1      1  Established        Grocery
1        2      1  Established  Miscellaneous
2        3     56        Local         Bakery
3        4     56        Local        Grocery
4        5     56        Local        Grocery
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74066 entries, 0 to 74065
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_id     74066 non-null  int64 
 1   brand       74066 non-null  int64 
 2   brand_type  74066 non-null  object
 3   category    74066 non-null  object
dtypes: int64(2), object(2)
memory usage: 2.3+ MB
None


In [14]:
len(item.brand.unique())

5528

In [15]:
print(item.brand_type.unique())
print(" ")
print(item.category.value_counts())

['Established' 'Local']
 
Grocery                   32448
Pharmaceutical            24471
Natural Products           2533
Dairy, Juices & Snacks     2425
Skin & Hair Care           2244
Meat                       2080
Packaged Meat              1966
Prepared Food              1880
Bakery                     1679
Seafood                     728
Flowers & Plants            664
Miscellaneous               385
Alcohol                     303
Garden                      102
Restauarant                  78
Salads                       40
Travel                       19
Fuel                         14
Vegetables (cut)              7
Name: category, dtype: int64


### Customer Demographics 

In [16]:
demo.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,NaN,4
1,6,46-55,Married,0,2,NaN,5
2,7,26-35,NaN,0,3,1,3
3,8,26-35,NaN,0,4,2,6
4,10,46-55,Single,0,1,NaN,5


In [17]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  431 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    object
 5   no_of_children  222 non-null    object
 6   income_bracket  760 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 41.7+ KB


In [18]:
demo.family_size.value_counts()

2     303
1     248
3     104
5+     57
4      48
Name: family_size, dtype: int64

In [19]:
import re
demo.no_of_children.fillna('0',inplace=True)
demo.marital_status.fillna("Not Known",inplace = True)
demo['family_size'] = demo.family_size.apply(lambda x: int(re.sub('\+','',x)))
demo['no_of_children'] = demo.no_of_children.apply(lambda x: int(re.sub('\+','',x)))
demo.loc[(demo.marital_status == "Not Known") & (demo.family_size==1),'marital_status']='Single'

In [20]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  760 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    int64 
 5   no_of_children  760 non-null    int64 
 6   income_bracket  760 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 41.7+ KB


In [21]:
demo.marital_status.value_counts()

Married      317
Single       297
Not Known    146
Name: marital_status, dtype: int64

In [22]:
demo.loc[(demo.marital_status == 'Not Known') & (demo.family_size - demo.no_of_children ==2),'marital_status']= 'Married'
demo.loc[(demo.marital_status == 'Not Known') & (demo.no_of_children == 0) & (demo.family_size ==2),'marital_status']= 'Married'

In [23]:
demo.marital_status.value_counts()

Married      450
Single       297
Not Known     13
Name: marital_status, dtype: int64

In [24]:
demo.head()

,customer_id,age_range,marital_status,rented,family_size,no_of_children,income_bracket
0,1,70+,Married,0,2,0,4
1,6,46-55,Married,0,2,0,5
2,7,26-35,Married,0,3,1,3
3,8,26-35,Married,0,4,2,6
4,10,46-55,Single,0,1,0,5


In [25]:
train.head()

,id,campaign_id,coupon_id,customer_id,redemption_status
0,1,13,27,1053,0
1,2,13,116,48,0
2,6,9,635,205,0
3,7,13,644,1050,0
4,9,8,1017,1489,0


In [26]:
camp_data.head()

,campaign_id,campaign_type,start_date,end_date,Days
0,24,Y,2013-10-21,2013-12-20,60
1,25,Y,2013-10-21,2013-11-22,32
2,20,Y,2013-09-07,2013-11-16,70
3,23,Y,2013-10-08,2013-11-15,38
4,21,Y,2013-09-16,2013-10-18,32


In [27]:
coupon.head()


,coupon_id,item_id
0,105,37
1,107,75
2,494,76
3,522,77
4,518,77


In [28]:
item.head(5)

,item_id,brand,brand_type,category
0,1,1,Established,Grocery
1,2,1,Established,Miscellaneous
2,3,56,Local,Bakery
3,4,56,Local,Grocery
4,5,56,Local,Grocery


In [29]:
trans.head(5)

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0
3,2012-01-02,1501,33647,1,67.32,0.00,0.0
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0


## Data Aggregation

In [30]:
coupon_used = []
for i,disc in enumerate(trans.coupon_discount):
    if disc != 0 :
        coupon_used.append(1)
    else:
        coupon_used.append(0)
trans['trans_with_coupon'] = pd.Series(coupon_used)
trans['count'] = 1
trans.head()

,date,customer_id,item_id,quantity,selling_price,other_discount,coupon_discount,trans_with_coupon,count
0,2012-01-02,1501,26830,1,35.26,-10.69,0.0,0,1
1,2012-01-02,1501,54253,1,53.43,-13.89,0.0,0,1
2,2012-01-02,1501,31962,1,106.50,-14.25,0.0,0,1
3,2012-01-02,1501,33647,1,67.32,0.00,0.0,0,1
4,2012-01-02,1501,48199,1,71.24,-28.14,0.0,0,1


Adding Features so that I could link data to other dataframes with item_id as common factor


### Variables used 

|Variable| value of variable   |
|------|-----------------------|
|  ct_cust | number of distinct customer who buy that item |
|------|---------------------------------------------------|
|  avg_order_quant | average order quantity |
|------|------------------------------------|
|  avg_price | average price |
|------|---------------------|
|  avg_oth_dis | average other discount |
|------|--------------------------------|
|  avg_cp_dis | average coupon discount |
|------|--------------------------------|
|  sum_cp_dis | sum of coupon discount |
|------|-------------------------------|
|  sum_oth_dis | sum of other discount |
|------|-------------------------------|
|sum_order_qnt | total order quantity |
|------|------------------------------|
|sum_total_dis | total discount |
|------|------------------------|
|avg_total_dis | average total discount |
|------|--------------------------------|
|sum_order_qnt | total order quantity |
|------|------------------------------|
|ct_ems | number od items applied per coupon |
|------|-------------------------------------|
|ct_brands | count of brands per coupons |
|------|---------------------------------|
|mode_cat | most frequent category of coupon |
|------------






Make a new table consisting of above mentioned variables using pd.pivot() with item_id as index

In [31]:
new_trans = pd.pivot_table(trans,index = 'item_id',
                           values=['customer_id','quantity','selling_price','other_discount','coupon_discount','count','trans_with_coupon'],
                           aggfunc = {'customer_id': lambda i:len(set(i)),
                                      'quantity' : np.mean,
                                      'selling_price':np.mean,
                                      'other_discount':np.mean,
                                      'coupon_discount':np.mean,
                                      'count':sum,
                                      'trans_with_coupon':sum
                                       }
                          )

In [32]:
new_cols = {'customer_id':'ct_cust',
            'quantity':'avg_order_qnt',
            'selling_price':'avg_price',
            'other_discount':'avg_oth_dis',
            'coupon_discount':'avg_cp_dis',
            'count':'ct_ord',
           'trans_with_coupon':'sum_cp_used'}
new_trans.rename(columns=new_cols, inplace = True)

In [33]:
new_trans.head()

,ct_ord,avg_cp_dis,ct_cust,avg_oth_dis,avg_order_qnt,avg_price,sum_cp_used
item_id,,,,,,,
1,2,0.0,2,0.0,1.0,124.31,0
2,1,0.0,1,0.0,1.0,35.26,0
3,1,0.0,1,0.0,1.0,56.64,0
4,1,0.0,1,0.0,1.0,54.85,0
5,1,0.0,1,0.0,1.0,81.57,0


In [34]:
new_trans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74063 entries, 1 to 74066
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ct_ord         74063 non-null  int64  
 1   avg_cp_dis     74063 non-null  float64
 2   ct_cust        74063 non-null  int64  
 3   avg_oth_dis    74063 non-null  float64
 4   avg_order_qnt  74063 non-null  float64
 5   avg_price      74063 non-null  float64
 6   sum_cp_used    74063 non-null  int64  
dtypes: float64(4), int64(3)
memory usage: 4.5 MB


In [35]:
new_trans['sum_cp_dis'] = new_trans['avg_cp_dis']*new_trans['ct_ord']
new_trans['sum_oth_dis'] = new_trans['avg_oth_dis']*new_trans['ct_ord']
new_trans['sum_order_qnt']= new_trans['avg_order_qnt']*new_trans['ct_ord']
new_trans['sum_total_dis'] = new_trans['sum_cp_dis'] +new_trans['sum_oth_dis']
new_trans['avg_total_dis'] = new_trans['avg_cp_dis'] +new_trans['avg_oth_dis']

In [36]:
new_trans.head()

,ct_ord,avg_cp_dis,ct_cust,avg_oth_dis,avg_order_qnt,avg_price,sum_cp_used,sum_cp_dis,sum_oth_dis,sum_order_qnt,sum_total_dis,avg_total_dis
item_id,,,,,,,,,,,,
1,2,0.0,2,0.0,1.0,124.31,0,0.0,0.0,2.0,0.0,0.0
2,1,0.0,1,0.0,1.0,35.26,0,0.0,0.0,1.0,0.0,0.0
3,1,0.0,1,0.0,1.0,56.64,0,0.0,0.0,1.0,0.0,0.0
4,1,0.0,1,0.0,1.0,54.85,0,0.0,0.0,1.0,0.0,0.0
5,1,0.0,1,0.0,1.0,81.57,0,0.0,0.0,1.0,0.0,0.0


Merge this transaction with coupon item mapping data frame 

In [37]:
new_trans_item = pd.merge(new_trans,item,on='item_id',how='left')
new_trans_item.head()

,item_id,ct_ord,avg_cp_dis,ct_cust,avg_oth_dis,avg_order_qnt,avg_price,sum_cp_used,sum_cp_dis,sum_oth_dis,sum_order_qnt,sum_total_dis,avg_total_dis,brand,brand_type,category
0,1,2,0.0,2,0.0,1.0,124.31,0,0.0,0.0,2.0,0.0,0.0,1,Established,Grocery
1,2,1,0.0,1,0.0,1.0,35.26,0,0.0,0.0,1.0,0.0,0.0,1,Established,Miscellaneous
2,3,1,0.0,1,0.0,1.0,56.64,0,0.0,0.0,1.0,0.0,0.0,56,Local,Bakery
3,4,1,0.0,1,0.0,1.0,54.85,0,0.0,0.0,1.0,0.0,0.0,56,Local,Grocery
4,5,1,0.0,1,0.0,1.0,81.57,0,0.0,0.0,1.0,0.0,0.0,56,Local,Grocery


Merge Coupon Data dataframe with above resultant 

In [38]:
coupon_newtrans_item = pd.merge(coupon,new_trans_item,on='item_id',how='left')
coupon_newtrans_item.head()

,coupon_id,item_id,ct_ord,avg_cp_dis,ct_cust,avg_oth_dis,avg_order_qnt,avg_price,sum_cp_used,sum_cp_dis,sum_oth_dis,sum_order_qnt,sum_total_dis,avg_total_dis,brand,brand_type,category
0,105,37,7.0,0.0,2.0,-17.557143,2.285714,113.120000,0.0,0.0,-122.90,16.0,-122.90,-17.557143,56.0,Local,Grocery
1,107,75,33.0,0.0,4.0,-5.159394,1.363636,47.787273,0.0,0.0,-170.26,45.0,-170.26,-5.159394,56.0,Local,Grocery
2,494,76,1.0,0.0,1.0,-17.450000,1.000000,89.050000,0.0,0.0,-17.45,1.0,-17.45,-17.450000,209.0,Established,Grocery
3,522,77,2.0,0.0,2.0,-16.920000,1.000000,70.350000,0.0,0.0,-33.84,2.0,-33.84,-16.920000,278.0,Established,Grocery
4,518,77,2.0,0.0,2.0,-16.920000,1.000000,70.350000,0.0,0.0,-33.84,2.0,-33.84,-16.920000,278.0,Established,Grocery


In [39]:
cp_trans_item1 = pd.pivot_table(coupon_newtrans_item, index ='coupon_id',
                                values=['item_id', 'coupon_id', 'brand', 'brand_type', 'category', 'ct_ord',
                                        'avg_cp_dis', 'sum_cp_used', 'ct_cust', 'avg_oth_dis', 'avg_order_qnt',
                                       'avg_price', 'sum_cp_dis', 'sum_oth_dis', 'sum_order_qnt',
                                       'sum_total_dis', 'avg_total_dis'],
                               aggfunc={
                                   'item_id': lambda x: len(set(x)),
                                   'brand': lambda x: len(set(x)),
                                   'brand_type': lambda x: x.unique()[0] if (len(x.unique())==1) else 'Both',
                                   'category':lambda x: x.mode()[0],
                                   'ct_ord': np.sum,
                                   'avg_cp_dis':np.mean,
                                   'sum_cp_used':np.sum,
                                   'ct_cust':np.sum,
                                   'avg_oth_dis':np.mean,
                                   'avg_order_qnt':np.mean,
                                   'ct_ord':np.sum,
                                   'avg_price':np.mean,
                                   'sum_cp_dis':np.sum,
                                   'sum_oth_dis':np.sum,
                                   'sum_order_qnt':np.sum,
                                   'sum_total_dis':np.sum,
                                   'avg_total_dis':np.mean
                               }
                              )

new_cols= {
                   'item_id': 'ct_items',#item count
                   'brand': 'ct_brands',
                   'category':'mode_cat'
    
                                  }
cp_trans_item1.rename(columns = new_cols, inplace = True)

In [40]:
cp_trans_item1.head()

,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,avg_total_dis,ct_brands,brand_type,mode_cat,ct_cust,ct_ord,ct_items,sum_cp_dis,sum_cp_used,sum_order_qnt,sum_oth_dis,sum_total_dis
coupon_id,,,,,,,,,,,,,,,,
1,-1.321524,1.221644,-25.273218,100.195701,-26.594742,3.0,Established,Natural Products,577.0,826.0,39,-1526.31,35.0,1034.0,-24468.37,-25994.68
2,0.000000,1.137500,-23.314062,114.354438,-23.314062,1.0,Established,Grocery,30.0,81.0,2,0.00,0.0,103.0,-1478.75,-1478.75
3,-2.747358,1.121525,-17.543797,129.981640,-20.291155,2.0,Established,Grocery,146.0,212.0,17,-578.81,15.0,248.0,-4952.62,-5531.43
4,-1.485152,1.020872,-37.411408,178.722897,-38.896560,1.0,Established,Grocery,536.0,676.0,24,-881.59,44.0,702.0,-26804.05,-27685.64
5,0.000000,1.000000,-27.265786,376.704214,-27.265786,1.0,Established,Pharmaceutical,42.0,44.0,7,0.00,0.0,44.0,-1228.88,-1228.88


In [41]:
cp_trans_item1['ct_cat']= pd.pivot_table(coupon_newtrans_item, 
                                         index='coupon_id',
                                         values ='category',
                                         aggfunc =lambda x: len(set(x)))
cp_trans_item1.head()

,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,avg_total_dis,ct_brands,brand_type,mode_cat,ct_cust,ct_ord,ct_items,sum_cp_dis,sum_cp_used,sum_order_qnt,sum_oth_dis,sum_total_dis,ct_cat
coupon_id,,,,,,,,,,,,,,,,,
1,-1.321524,1.221644,-25.273218,100.195701,-26.594742,3.0,Established,Natural Products,577.0,826.0,39,-1526.31,35.0,1034.0,-24468.37,-25994.68,2
2,0.000000,1.137500,-23.314062,114.354438,-23.314062,1.0,Established,Grocery,30.0,81.0,2,0.00,0.0,103.0,-1478.75,-1478.75,1
3,-2.747358,1.121525,-17.543797,129.981640,-20.291155,2.0,Established,Grocery,146.0,212.0,17,-578.81,15.0,248.0,-4952.62,-5531.43,1
4,-1.485152,1.020872,-37.411408,178.722897,-38.896560,1.0,Established,Grocery,536.0,676.0,24,-881.59,44.0,702.0,-26804.05,-27685.64,1
5,0.000000,1.000000,-27.265786,376.704214,-27.265786,1.0,Established,Pharmaceutical,42.0,44.0,7,0.00,0.0,44.0,-1228.88,-1228.88,1


Merge with training data set 

In [42]:
train_cp_trans_item = pd.merge(train, cp_trans_item1, on = 'coupon_id', how= 'left')
train_cp_trans_item.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,avg_total_dis,...,mode_cat,ct_cust,ct_ord,ct_items,sum_cp_dis,sum_cp_used,sum_order_qnt,sum_oth_dis,sum_total_dis,ct_cat
0,1,13,27,1053,0,-1.241694,1.169768,-51.732305,84.478583,-52.974000,...,Grocery,2403.0,3713.0,125,-5349.93,191.0,4661.0,-225584.38,-230934.31,1
1,2,13,116,48,0,0.000000,1.050926,-10.661343,63.833519,-10.661343,...,Grocery,61.0,75.0,3,0.00,0.0,86.0,-684.27,-684.27,1
2,6,9,635,205,0,-2.987321,1.353802,-142.498493,332.128380,-145.485813,...,Pharmaceutical,118.0,142.0,67,-605.54,15.0,202.0,-23566.35,-24171.89,1
3,7,13,644,1050,0,-4.452500,1.110390,-5.010601,94.261640,-9.463101,...,Grocery,14.0,39.0,4,-17.81,1.0,47.0,-309.91,-327.72,1
4,9,8,1017,1489,0,-0.198055,1.071044,-33.629121,181.621631,-33.827176,...,Grocery,874.0,1420.0,32,-569.92,23.0,1610.0,-54950.74,-55520.66,1


In [43]:
cust_trans = pd.pivot_table(trans, index = 'customer_id',
                           values =['item_id', 'quantity', 'selling_price',
                                    'other_discount','coupon_discount','count','trans_with_coupon'],
                           aggfunc={'item_id':len, 
                                    'quantity':np.mean,
                                    'selling_price':np.mean,
                                    'other_discount':np.mean,
                                    'coupon_discount':np.mean,
                                   'trans_with_coupon':np.mean})
new_cols={'item_id':'ct_items_cust',
          'quantity':'avg_qnt_cust',
          'selling_price':'avg_price_cust',
          'other_discount':'avg_oth_disc_cust',
          'coupon_discount':'avg_cp_disc_cust',
          'trans_with_coupon':'avg_cp_use_cust'}


cust_trans.rename(columns=new_cols, inplace= True)

In [44]:
cust_trans['sum_qnt_cust']=cust_trans['ct_items_cust']*cust_trans['avg_qnt_cust']
cust_trans['sum_price_cust']=cust_trans['ct_items_cust']*cust_trans['avg_price_cust']
cust_trans['sum_oth_disc_cust']=cust_trans['ct_items_cust']*cust_trans['avg_oth_disc_cust']
cust_trans['sum_cp_disc_cust']=cust_trans['ct_items_cust']*cust_trans['avg_cp_disc_cust']
cust_trans['ct_cp_use_cust']=cust_trans['ct_items_cust']*cust_trans['avg_cp_use_cust']

In [45]:
cust_trans.head()

,avg_cp_disc_cust,ct_items_cust,avg_oth_disc_cust,avg_qnt_cust,avg_price_cust,avg_cp_use_cust,sum_qnt_cust,sum_price_cust,sum_oth_disc_cust,sum_cp_disc_cust,ct_cp_use_cust
customer_id,,,,,,,,,,,
1,-2.019876,1048,-16.250382,1.170802,94.001842,0.074427,1227.0,98513.93,-17030.40,-2116.83,78.0
2,-0.595084,419,-16.830430,1.131265,102.864033,0.009547,474.0,43100.03,-7051.95,-249.34,4.0
3,-3.091546,705,-22.714227,11.578723,103.617404,0.075177,8163.0,73050.27,-16013.53,-2179.54,53.0
4,-0.404773,220,-13.305409,1.272727,154.423727,0.004545,280.0,33973.22,-2927.19,-89.05,1.0
5,-0.114684,792,-13.657917,117.869949,130.827146,0.002525,93353.0,103615.10,-10817.07,-90.83,2.0


In [46]:
cust_trans = pd.merge(cust_trans, demo, on ='customer_id', how ='left')

In [47]:
train_cp_trans_item_cust = pd.merge(train_cp_trans_item, cust_trans, on='customer_id', how='left')

In [48]:
train_f = pd.merge(train_cp_trans_item_cust, camp_data, on='campaign_id', how='left')

In [49]:
train_f.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,avg_total_dis,...,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_type,start_date,end_date,Days
0,1,13,27,1053,0,-1.241694,1.169768,-51.732305,84.478583,-52.974000,...,46-55,Single,0.0,1.0,0.0,5.0,X,2013-05-19,2013-07-05,47
1,2,13,116,48,0,0.000000,1.050926,-10.661343,63.833519,-10.661343,...,36-45,Married,0.0,2.0,0.0,3.0,X,2013-05-19,2013-07-05,47
2,6,9,635,205,0,-2.987321,1.353802,-142.498493,332.128380,-145.485813,...,46-55,Married,0.0,2.0,0.0,7.0,Y,2013-03-11,2013-04-12,32
3,7,13,644,1050,0,-4.452500,1.110390,-5.010601,94.261640,-9.463101,...,NaN,NaN,NaN,NaN,NaN,NaN,X,2013-05-19,2013-07-05,47
4,9,8,1017,1489,0,-0.198055,1.071044,-33.629121,181.621631,-33.827176,...,46-55,Married,0.0,2.0,0.0,3.0,X,2013-02-16,2013-04-05,48


Remove Null values by inserting their modes int NaN values

In [50]:
train_f.isna().sum()

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
avg_cp_dis               0
avg_order_qnt            0
avg_oth_dis              0
avg_price                0
avg_total_dis            0
ct_brands                0
brand_type               0
mode_cat                 0
ct_cust                  0
ct_ord                   0
ct_items                 0
sum_cp_dis               0
sum_cp_used              0
sum_order_qnt            0
sum_oth_dis              0
sum_total_dis            0
ct_cat                   0
avg_cp_disc_cust         0
ct_items_cust            0
avg_oth_disc_cust        0
avg_qnt_cust             0
avg_price_cust           0
avg_cp_use_cust          0
sum_qnt_cust             0
sum_price_cust           0
sum_oth_disc_cust        0
sum_cp_disc_cust         0
ct_cp_use_cust           0
age_range            34708
marital_status       34708
rented               34708
family_size          34708
n

In [51]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   customer_id     760 non-null    int64 
 1   age_range       760 non-null    object
 2   marital_status  760 non-null    object
 3   rented          760 non-null    int64 
 4   family_size     760 non-null    int64 
 5   no_of_children  760 non-null    int64 
 6   income_bracket  760 non-null    int64 
dtypes: int64(5), object(2)
memory usage: 41.7+ KB


In [52]:
from scipy.stats import mode
for i in demo.columns.tolist()[1:]:
    train_f[i].fillna(mode(train_f[i]).mode[0],inplace=True)
train_f.isna().sum() 

id                   0
campaign_id          0
coupon_id            0
customer_id          0
redemption_status    0
avg_cp_dis           0
avg_order_qnt        0
avg_oth_dis          0
avg_price            0
avg_total_dis        0
ct_brands            0
brand_type           0
mode_cat             0
ct_cust              0
ct_ord               0
ct_items             0
sum_cp_dis           0
sum_cp_used          0
sum_order_qnt        0
sum_oth_dis          0
sum_total_dis        0
ct_cat               0
avg_cp_disc_cust     0
ct_items_cust        0
avg_oth_disc_cust    0
avg_qnt_cust         0
avg_price_cust       0
avg_cp_use_cust      0
sum_qnt_cust         0
sum_price_cust       0
sum_oth_disc_cust    0
sum_cp_disc_cust     0
ct_cp_use_cust       0
age_range            0
marital_status       0
rented               0
family_size          0
no_of_children       0
income_bracket       0
campaign_type        0
start_date           0
end_date             0
Days                 0
dtype: int6

### calclulating correlation of every variables to reduce overfitting/underfitting

In [53]:
def correlation(df,t):
    mat = df.corr()
    for column in mat.columns:
        if(sum(abs(mat[column]>t))>1):
            print('the correlation is higher than {} with {}'.format(t, column),
                  '\n',mat.loc[abs(mat[column])>t][column],'\n')

In [54]:
correlation(train_f,0.7)

the correlation is higher than 0.7 with avg_oth_dis 
 avg_oth_dis      1.000000
avg_total_dis    0.978775
Name: avg_oth_dis, dtype: float64 

the correlation is higher than 0.7 with avg_total_dis 
 avg_oth_dis      0.978775
avg_total_dis    1.000000
Name: avg_total_dis, dtype: float64 

the correlation is higher than 0.7 with ct_brands 
 ct_brands        1.000000
ct_items         0.719483
sum_oth_dis     -0.737065
sum_total_dis   -0.734667
Name: ct_brands, dtype: float64 

the correlation is higher than 0.7 with ct_cust 
 ct_cust          1.000000
ct_ord           0.988502
ct_items         0.828651
sum_cp_dis      -0.881829
sum_cp_used      0.922261
sum_order_qnt    0.986603
sum_oth_dis     -0.948307
sum_total_dis   -0.951566
ct_cat           0.769492
Name: ct_cust, dtype: float64 

the correlation is higher than 0.7 with ct_ord 
 ct_cust          0.988502
ct_ord           1.000000
ct_items         0.765948
sum_cp_dis      -0.854871
sum_cp_used      0.915109
sum_order_qnt    0.999084
s

Drop ever columns which has wither extremely high or very low correlation 

In [55]:
train_f.columns
dropped_cols = ['avg_total_dis', 'mode_cat', 'ct_cust',
       'ct_ord', 'ct_items', 'sum_cp_dis', 'sum_order_qnt',
       'sum_oth_dis', 'sum_total_dis', 'avg_cp_disc_cust',
       'ct_items_cust',
       'sum_qnt_cust', 
       'sum_oth_disc_cust', 'sum_cp_disc_cust', 'ct_cp_use_cust','start_date', 'end_date']
train_f1 = train_f.drop(columns = dropped_cols)
train_f1.head()

,id,campaign_id,coupon_id,customer_id,redemption_status,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,ct_brands,...,avg_cp_use_cust,sum_price_cust,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_type,Days
0,1,13,27,1053,0,-1.241694,1.169768,-51.732305,84.478583,2.0,...,0.003226,57120.75,46-55,Single,0.0,1.0,0.0,5.0,X,47
1,2,13,116,48,0,0.000000,1.050926,-10.661343,63.833519,1.0,...,0.031169,90185.10,36-45,Married,0.0,2.0,0.0,3.0,X,47
2,6,9,635,205,0,-2.987321,1.353802,-142.498493,332.128380,1.0,...,0.087629,117461.66,46-55,Married,0.0,2.0,0.0,7.0,Y,32
3,7,13,644,1050,0,-4.452500,1.110390,-5.010601,94.261640,1.0,...,0.008439,23291.42,46-55,Married,0.0,2.0,0.0,5.0,X,47
4,9,8,1017,1489,0,-0.198055,1.071044,-33.629121,181.621631,1.0,...,0.017794,67797.49,46-55,Married,0.0,2.0,0.0,3.0,X,48


### Model Fitting 

In [56]:
from sklearn.metrics import classification_report,roc_auc_score,roc_curve
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [60]:
test_id = train_f1['id']
test = train_f1['redemption_status']
train_f1.drop(['id','campaign_id','redemption_status'], axis=1, inplace=True)

In [61]:
train_f1.head()

,coupon_id,customer_id,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,ct_brands,brand_type,sum_cp_used,ct_cat,...,avg_cp_use_cust,sum_price_cust,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_type,Days
0,27,1053,-1.241694,1.169768,-51.732305,84.478583,2.0,Established,191.0,1,...,0.003226,57120.75,46-55,Single,0.0,1.0,0.0,5.0,X,47
1,116,48,0.000000,1.050926,-10.661343,63.833519,1.0,Local,0.0,1,...,0.031169,90185.10,36-45,Married,0.0,2.0,0.0,3.0,X,47
2,635,205,-2.987321,1.353802,-142.498493,332.128380,1.0,Established,15.0,1,...,0.087629,117461.66,46-55,Married,0.0,2.0,0.0,7.0,Y,32
3,644,1050,-4.452500,1.110390,-5.010601,94.261640,1.0,Established,1.0,1,...,0.008439,23291.42,46-55,Married,0.0,2.0,0.0,5.0,X,47
4,1017,1489,-0.198055,1.071044,-33.629121,181.621631,1.0,Established,23.0,1,...,0.017794,67797.49,46-55,Married,0.0,2.0,0.0,3.0,X,48


In [66]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_f1["marital_status"] = encoder.fit_transform(train_f1["marital_status"])
train_f1["age_range"]= encoder.fit_transform(train_f1["age_range"])
train_f1["campaign_type"]=encoder.fit_transform(train_f1["campaign_type"])
train_f1["brand_type"]=encoder.fit_transform(train_f1["brand_type"])


In [67]:
train_f1.head()

,coupon_id,customer_id,avg_cp_dis,avg_order_qnt,avg_oth_dis,avg_price,ct_brands,brand_type,sum_cp_used,ct_cat,...,avg_cp_use_cust,sum_price_cust,age_range,marital_status,rented,family_size,no_of_children,income_bracket,campaign_type,Days
0,27,1053,-1.241694,1.169768,-51.732305,84.478583,2.0,1,191.0,1,...,0.003226,57120.75,3,2,0.0,1.0,0.0,5.0,0,47
1,116,48,0.000000,1.050926,-10.661343,63.833519,1.0,2,0.0,1,...,0.031169,90185.10,2,0,0.0,2.0,0.0,3.0,0,47
2,635,205,-2.987321,1.353802,-142.498493,332.128380,1.0,1,15.0,1,...,0.087629,117461.66,3,0,0.0,2.0,0.0,7.0,1,32
3,644,1050,-4.452500,1.110390,-5.010601,94.261640,1.0,1,1.0,1,...,0.008439,23291.42,3,0,0.0,2.0,0.0,5.0,0,47
4,1017,1489,-0.198055,1.071044,-33.629121,181.621631,1.0,1,23.0,1,...,0.017794,67797.49,3,0,0.0,2.0,0.0,3.0,0,48


In [68]:
train_f1.columns

Index(['coupon_id', 'customer_id', 'avg_cp_dis', 'avg_order_qnt',
       'avg_oth_dis', 'avg_price', 'ct_brands', 'brand_type', 'sum_cp_used',
       'ct_cat', 'avg_oth_disc_cust', 'avg_qnt_cust', 'avg_price_cust',
       'avg_cp_use_cust', 'sum_price_cust', 'age_range', 'marital_status',
       'rented', 'family_size', 'no_of_children', 'income_bracket',
       'campaign_type', 'Days'],
      dtype='object')

In [70]:
x = train_f1
y = test 
cols = ['coupon_id', 'customer_id', 'avg_cp_dis', 'avg_order_qnt',
       'avg_oth_dis', 'avg_price', 'ct_brands', 'brand_type', 'sum_cp_used',
       'ct_cat', 'avg_oth_disc_cust', 'avg_qnt_cust', 'avg_price_cust',
       'avg_cp_use_cust', 'sum_price_cust', 'age_range', 'marital_status',
       'rented', 'family_size', 'no_of_children', 'income_bracket',
       'campaign_type', 'Days']
feat = x[cols]
scaler = StandardScaler().fit(feat.values)
feat = scaler.transform(feat.values)
x[cols] = feat
x = np.array(x)

In [71]:
x

array([[-1.63461265,  0.58131177,  0.17613832, ...,  0.06279887,
        -0.59882521,  0.36264341],
       [-1.36488621, -1.61873523,  0.38967513, ..., -1.07854721,
        -0.59882521,  0.36264341],
       [ 0.2080129 , -1.27504629, -0.12406082, ...,  1.20414494,
         1.66993636, -2.0781127 ],
       ...,
       [ 0.56865836,  0.47404579, -0.1674324 , ...,  0.06279887,
        -0.59882521,  0.52536048],
       [-1.3103348 , -1.56838589,  0.38967513, ..., -0.50787417,
        -0.59882521,  0.36264341],
       [ 0.34742207, -0.36000187,  0.0421844 , ...,  0.06279887,
        -0.59882521,  0.36264341]])

In [72]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=None)

In [73]:
x_train.shape,y_train.shape,x_test.shape,y_test.shape

((54858, 23), (54858,), (23511, 23), (23511,))

In [75]:
model1 = LogisticRegression()
model1.fit(x_train,y_train)
pred_lr = model1.predict(x_test)
roc_auc_score(y_test,pred_lr)

0.5022307828890336

In [80]:
from sklearn.naive_bayes import GaussianNB
model2 = GaussianNB()
model2.fit(x_train,y_train)
pred_nb = model2.predict(x_test)
print(roc_auc_score(y_test,pred_nb))

0.5739451289496375
